<a href="https://colab.research.google.com/github/A-sy-coding/ML-Class/blob/master/card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rc('font',family='MalGun Gothic')
mpl.rc('axes',unicode_minus = False)

In [ ]:
import os
os.getcwd()
os.chdir('/content/drive/MyDrive/card')

In [ ]:
# 데이터 불러오기
card = pd.read_csv('train.csv')
card_test = pd.read_csv('test.csv')

In [ ]:
# index와 occyp_type 변수를 제거한다.
card = card.drop(columns=['index','occyp_type','FLAG_MOBIL'])

card_test = card_test.drop(columns=['index','occyp_type','FLAG_MOBIL'])

card.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,2.0,-26.0,2.0


# 데이터 전처리

In [ ]:
# 각 특성들의 값의 개수 출력
category_count = card.apply(lambda x : len(x.unique())) 

# 특성의 개수가 2개인 것, 2개보다 크고 10개보다 작은 것 , 10개보다 큰것으로 세 범주로 나누어 본다.

group1 = category_count[category_count == 2].index
group2 = category_count[(category_count > 2) & (category_count <= 10)].index
group3 = category_count[category_count > 10].index


#################################
# test data 전처리

category_count_test = card_test.apply(lambda x : len(x.unique())) 

# 특성의 개수가 2개인 것, 2개보다 크고 10개보다 작은 것 , 10개보다 큰것으로 세 범주로 나누어 본다.

group1_test = category_count_test[category_count_test == 2].index
group2_test = category_count_test[(category_count_test > 2) & (category_count_test <= 10)].index
group3_test = category_count_test[category_count_test > 10].index

# ==========================================================

* 두개의 값들을 가지는 변수 변경

In [ ]:
# 이진변수만 존재한다.
card[group1]

,gender,car,reality,work_phone,phone,email
0,F,N,N,0,0,0
1,F,N,Y,0,0,1
2,M,Y,Y,0,1,0
3,F,N,Y,0,1,0
4,F,Y,Y,0,0,0
...,...,...,...,...,...,...
26452,F,N,N,0,0,0
26453,F,N,Y,0,0,0
26454,F,Y,N,0,0,0
26455,M,N,Y,0,0,0


In [ ]:
# 문자형 변수들을 이진값으로 바꾸어준다.
card['gender'] = card['gender'].replace(['F','M'],[0,1])
card['car'] = card['car'].replace(['N','Y'],[0,1])
card['reality'] = card['reality'].replace(['N','Y'],[0,1])



#########################################

# test 데이터도 전처리 적용
card_test['gender'] = card_test['gender'].replace(['F','M'],[0,1])
card_test['car'] = card_test['car'].replace(['N','Y'],[0,1])
card_test['reality'] = card_test['reality'].replace(['N','Y'],[0,1])

# ==========================================================

* 3개 이상의 값을 가지고 있는 변수 변경

In [ ]:
# 2개보다 크고 10개보다 작은 범주 확인
card[group2]

,child_num,income_type,edu_type,family_type,house_type,family_size,credit
0,0,Commercial associate,Higher education,Married,Municipal apartment,2.0,1.0
1,1,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,3.0,1.0
2,0,Working,Higher education,Married,House / apartment,2.0,2.0
3,0,Commercial associate,Secondary / secondary special,Married,House / apartment,2.0,0.0
4,0,State servant,Higher education,Married,House / apartment,2.0,2.0
...,...,...,...,...,...,...,...
26452,2,State servant,Secondary / secondary special,Married,House / apartment,4.0,1.0
26453,1,Working,Higher education,Separated,House / apartment,2.0,2.0
26454,0,Working,Secondary / secondary special,Civil marriage,With parents,2.0,2.0
26455,0,Working,Incomplete higher,Single / not married,House / apartment,1.0,2.0


In [ ]:
card_test[group2_test]

,child_num,income_type,edu_type,family_type,house_type,family_size
0,0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,2.0
1,0,State servant,Higher education,Married,House / apartment,2.0
2,0,Working,Secondary / secondary special,Married,House / apartment,2.0
3,0,Commercial associate,Secondary / secondary special,Married,House / apartment,2.0
4,0,State servant,Higher education,Married,House / apartment,2.0
...,...,...,...,...,...,...
9995,0,Working,Incomplete higher,Married,House / apartment,2.0
9996,0,Working,Secondary / secondary special,Civil marriage,House / apartment,2.0
9997,0,Working,Secondary / secondary special,Married,House / apartment,2.0
9998,0,Commercial associate,Secondary / secondary special,Married,House / apartment,2.0


In [ ]:
# 3보다 큰 값들을 3으로 지정
card.loc[card['child_num']>3,'child_num'] = 3

# 결과 확인
card['child_num'].value_counts()

###########################################
# test data 전처리

# 3보다 큰 값들을 3으로 지정
card_test.loc[card_test['child_num']>3,'child_num'] = 3


* family_size 변수 확인하기

In [ ]:
# 4보다 큰 값들을 4로 지정한다.
card.loc[card['family_size'] >4,'family_size'] = 4

# 변경 결과 확인
card['family_size'].value_counts()

###########################################
# test data 전처리

# 4보다 큰 값들을 4로 지정한다.
card_test.loc[card_test['family_size'] >4,'family_size'] = 4

In [ ]:
# 나머지 문자형 변수 인코딩하기 --> labelencoding 이용
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer

#['income_type','edu_type','family_type','house_type']

# 원하는 열지정
encoder = LabelEncoder()
names = ['income_type','edu_type','family_type','house_type']

# 원하는 열만 인코딩하는 함수 만들기
def change_label(variable,names):
    
    for i in names:
        variable[i] = encoder.fit_transform(variable[i])

change_label(card,names)

############################
# test data 전처리

change_label(card_test,names)

# ==========================================================

* 10개 보다 큰 값들을 가지고 있는 변수 확인

In [ ]:
card[group3]

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,begin_month
0,202500.0,-13899,-4709,-6.0
1,247500.0,-11380,-1540,-5.0
2,450000.0,-19087,-4434,-22.0
3,202500.0,-15088,-2092,-37.0
4,157500.0,-15037,-2105,-26.0
...,...,...,...,...
26452,225000.0,-12079,-1984,-2.0
26453,180000.0,-15291,-2475,-47.0
26454,292500.0,-10082,-2015,-25.0
26455,171000.0,-10145,-107,-59.0


* income_total 변수 알아보기

In [ ]:
# income_total을 구간화 하기 위해서 각 값의 사분위 비율을 구해본다.
q_min = card['income_total'].describe()['min']
q1 = card['income_total'].describe()['25%']
q2 = card['income_total'].describe()['50%']
q3 = card['income_total'].describe()['75%']
q_max = card['income_total'].describe()['max']

#####################################################
# test data 전처리 

# income_total을 구간화 하기 위해서 각 값의 사분위 비율을 구해본다.
q_min_test = card_test['income_total'].describe()['min']
q1_test = card_test['income_total'].describe()['25%']
q2_test = card_test['income_total'].describe()['50%']
q3_test = card_test['income_total'].describe()['75%']
q_max_test = card_test['income_total'].describe()['max']

In [ ]:
# 구간 지정하기
bins = [q_min,q1,q2,q3,q_max]

############################################
# 구간을 나누어 주는 함수 만들기
def get_category(array,bins):
    category = pd.cut(array,bins = bins,include_lowest=True, labels= [i for i in range(1,len(bins))])
    return category
############################################

# 구간 변경한 값들을 새로운 변수로 추가하기
card['income_category'] = get_category(card['income_total'],bins)


######################################################
# test data 전처리
bins_test = [q_min_test,q1_test,q2_test,q3_test,q_max_test]
card_test['income_category'] = get_category(card_test['income_total'],bins_test)

* DAYS_BIRTH 변수 확인하기

In [ ]:
# DAYS_BIRTH-> 카드 발급일로부터의 출생일
card['DAYS_BIRTH'] 

########################
# 출생일을 이용해 나이 추출

card['DAYS_BIRTH'] = abs(card['DAYS_BIRTH'] ) # 양수로 변환

# 일년은 365일이고 일수를 365일로 나누어 몫을 구하면 년수를 구할 수 있다. --> 이때 년수는 나이가 될 수 있다.
card['age'] = card['DAYS_BIRTH'] // 365

# 각 나이를 범주화 시키기
# 20대 미만, 20대, 30대, 40대, 50대, 60대 이상

# 나이별로 범주형으로 변환시키는 함수 구현
def get_age_category(array):
    
    array.loc[array < 20,] = 1
    array.loc[(array >= 20) & (array < 30) , ] = 2
    array.loc[(array >= 30) & (array < 40) , ] = 3
    array.loc[(array >= 40) & (array < 50) , ] = 4
    array.loc[(array >= 50) & (array < 60) , ] = 5
    array.loc[array >= 60 , ] = 6

# 새로운 변수 생성
card['age_category'] = card['age']

# 함수 적용
get_age_category(card['age_category'])

# 변형 확인
card[['age','age_category']].head()


#######################################################
# test data  적용

card_test['DAYS_BIRTH'] = abs(card_test['DAYS_BIRTH'] ) # 양수로 변환

# 일년은 365일이고 일수를 365일로 나누어 몫을 구하면 년수를 구할 수 있다. --> 이때 년수는 나이가 될 수 있다.
card_test['age'] = card_test['DAYS_BIRTH'] // 365

# 새로운 변수 생성
card_test['age_category'] = card_test['age']

# 함수 적용
get_age_category(card_test['age_category'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [ ]:
# age_actegory에서 범위를 좀 더 합쳐보자 --> 더 넓게 잡는다.
card.loc[card['age_category']<4,'age_binary'] = 0
card.loc[card['age_category']>=4, 'age_binary' ]  =1


#########################
# test data에 적용

card_test.loc[card_test['age_category']<4,'age_binary'] = 0
card_test.loc[card_test['age_category']>=4, 'age_binary' ]  =1

* DAYS_EMPLOYED 변수 확인하기

In [ ]:
# 취업자와 비취업자의 변수를 새로 추가하기
# 고용일 이후 일자와 고용되지 않은 일자를 두개의 변수로 나누어서 고려한다.

# 새로운 변수 추가 -> 초기화
card['employ'] = 0
card['unemploy'] = 0

# 고용일과 비고용일이라는 새로운 변수 추가  --> 초기화
card['employ_day'] = 0
card['unemploy_day'] = 0

# 고용과 비고용의 두 개의 변수로 나누는 함수 구현
def new_variable_divide(array,employ,unemploy,employ_day,unemploy_day):
    
    # 인덱스값 저장
    mark_employ = np.where(array < 0 )[0][:]
    mark_unemploy = np.where(array > 0 )[0][:]
    
    # employ변수에서 취업했으면 1, unemploy에서 취업하지 않았으면 1
    card.loc[mark_employ,employ] = 1
    card.loc[mark_unemploy,unemploy] = 1
    
    # 음수이면 고용일 변수에 저장, 양수이면 비고용일 변수에 저장
    card.loc[mark_employ,employ_day] = card.loc[mark_employ,'DAYS_EMPLOYED'] * -1  # 양수로 변경
    card.loc[mark_unemploy,unemploy_day] = card.loc[mark_unemploy,'DAYS_EMPLOYED']
    
# 함수 실행하기
new_variable_divide(card['DAYS_EMPLOYED'],'employ','unemploy','employ_day','unemploy_day')

# 결과 확인하기
card.head()

##################################
# 고용되지 않는 상태의 경우는 일수가 너무 커서 그냥 0으로 취급

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit,income_category,age,age_category,age_binary,employ,unemploy,employ_day,unemploy_day
0,0,0,0,0,202500.0,0,1,1,2,13899,-4709,0,0,0,2.0,-6.0,1.0,3,38,3,0.0,1,0,4709,0
1,0,0,1,1,247500.0,0,4,0,1,11380,-1540,0,0,1,3.0,-5.0,1.0,4,31,3,0.0,1,0,1540,0
2,1,1,1,0,450000.0,4,1,1,1,19087,-4434,0,1,0,2.0,-22.0,2.0,4,52,5,1.0,1,0,4434,0
3,0,0,1,0,202500.0,0,4,1,1,15088,-2092,0,1,0,2.0,-37.0,0.0,3,41,4,1.0,1,0,2092,0
4,0,1,1,0,157500.0,2,1,1,1,15037,-2105,0,0,0,2.0,-26.0,2.0,2,41,4,1.0,1,0,2105,0


In [ ]:
# test data에 적용

# 새로운 변수 추가 -> 초기화
card_test['employ'] = 0
card_test['unemploy'] = 0

# 고용일과 비고용일이라는 새로운 변수 추가  --> 초기화
card_test['employ_day'] = 0
card_test['unemploy_day'] = 0

# 고용과 비고용의 두 개의 변수로 나누는 함수 구현
def new_variable_divide(array,employ,unemploy,employ_day,unemploy_day):
    
    # 인덱스값 저장
    mark_employ = np.where(array < 0 )[0][:]
    mark_unemploy = np.where(array > 0 )[0][:]
    
    # employ변수에서 취업했으면 1, unemploy에서 취업하지 않았으면 1
    card_test.loc[mark_employ,employ] = 1
    card_test.loc[mark_unemploy,unemploy] = 1
    
    # 음수이면 고용일 변수에 저장, 양수이면 비고용일 변수에 저장
    card_test.loc[mark_employ,employ_day] = card_test.loc[mark_employ,'DAYS_EMPLOYED'] * -1  # 양수로 변경
    card_test.loc[mark_unemploy,unemploy_day] = card_test.loc[mark_unemploy,'DAYS_EMPLOYED']
    
# 함수 실행하기
new_variable_divide(card_test['DAYS_EMPLOYED'],'employ','unemploy','employ_day','unemploy_day')

# 결과 확인하기
card_test.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,income_category,age,age_category,age_binary,employ,unemploy,employ_day,unemploy_day
0,1,1,0,0,112500.0,1,4,0,1,21990,365243,0,1,0,2.0,-60.0,1,60,6,1.0,0,1,0,365243
1,0,0,1,0,135000.0,2,1,1,1,18964,-8671,0,1,0,2.0,-36.0,2,51,5,1.0,1,0,8671,0
2,0,0,1,0,69372.0,4,4,1,1,15887,-217,1,1,0,2.0,-40.0,1,43,4,1.0,1,0,217,0
3,1,1,0,0,112500.0,0,4,1,1,19270,-2531,1,0,0,2.0,-41.0,1,52,5,1.0,1,0,2531,0
4,0,1,1,0,225000.0,2,1,1,1,17822,-9385,1,0,0,2.0,-8.0,3,48,4,1.0,1,0,9385,0


In [ ]:
# 고용되지 않는 상태는 0년차로 , 고용된지 1년이 안된 상태를 1년차로 두어서 각 년차에서 +1을 해준다.

# lambda 함수를 사용하여 위의 조건 만족시키고 고용년수 출력하기
employ_year = pd.DataFrame(map(lambda x: 0 if x == 0 else (x//365)+1, card['employ_day']))

# 새로운 변수에 추가
card['employ_year'] = employ_year

#################################################
# test data 적용

# lambda 함수를 사용하여 위의 조건 만족시키고 고용년수 출력하기
employ_year_test = pd.DataFrame(map(lambda x: 0 if x == 0 else (x//365)+1, card_test['employ_day']))

# 새로운 변수에 추가
card_test['employ_year'] = employ_year_test

In [ ]:
# employ_year를 구간을 나누어서 변수 추가시키기
card.loc[(card['employ_year']>30) ,'employ_year_category']= 2
card.loc[(card['employ_year']>10) & (card['employ_year']<=30) ,'employ_year_category']= 1
card.loc[(card['employ_year']<=10) ,'employ_year_category']= 0


#################
# test data에 적용

card_test.loc[(card_test['employ_year']>30) ,'employ_year_category']= 2
card_test.loc[(card_test['employ_year']>10) & (card_test['employ_year']<=30) ,'employ_year_category']= 1
card_test.loc[(card_test['employ_year']<=10) ,'employ_year_category']= 0

* begin_month 확인하기

In [ ]:
# 양수로 변경하기
card['begin_month'] =  card['begin_month']*-1

########################
# test data 
card_test['begin_month'] =  card_test['begin_month']*-1

In [ ]:
# 년도로 추가하기
card['begin_year'] = card['begin_month']//12

card['begin_year'].value_counts()

#############################
# test data
card_test['begin_year'] = card_test['begin_month']//12

# ==========================================================

# 변수 확인하고 제거하기

In [ ]:
# income_total과 DAYS_BIRTH, DAYS_EMPLOYED를 제거한다.   --> 결과를 확인하고 begin_month도 삭제 고려
# employ_day와 unemploy_day도 삭제 --> 전체를 년수로 맞춘다.

# 위의 조건들을 삭제한 변수
card_delete = card.drop(columns=['income_total','DAYS_BIRTH','DAYS_EMPLOYED','employ_day','unemploy_day','begin_month','unemploy','age'])
card_delete

#########################
# test data
card_test_delete = card_test.drop(columns=['income_total','DAYS_BIRTH','DAYS_EMPLOYED','employ_day','unemploy_day','begin_month','unemploy','age'])
card_test_delete

,gender,car,reality,child_num,income_type,edu_type,family_type,house_type,work_phone,phone,email,family_size,income_category,age_category,age_binary,employ,employ_year,employ_year_category,begin_year
0,1,1,0,0,1,4,0,1,0,1,0,2.0,1,6,1.0,0,0,0.0,5.0
1,0,0,1,0,2,1,1,1,0,1,0,2.0,2,5,1.0,1,24,1.0,3.0
2,0,0,1,0,4,4,1,1,1,1,0,2.0,1,4,1.0,1,1,0.0,3.0
3,1,1,0,0,0,4,1,1,1,0,0,2.0,1,5,1.0,1,7,0.0,3.0
4,0,1,1,0,2,1,1,1,1,0,0,2.0,3,4,1.0,1,26,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,0,4,2,1,1,1,1,0,2.0,3,5,1.0,1,15,1.0,1.0
9996,1,1,1,0,4,4,0,1,1,0,0,2.0,3,2,0.0,1,4,0.0,2.0
9997,0,0,1,0,4,4,1,1,0,0,0,2.0,4,5,1.0,1,39,2.0,4.0
9998,0,1,0,0,0,4,1,1,0,1,0,2.0,3,4,1.0,1,3,0.0,2.0


# ==========================================================

# 훈련데이터와 테스트 데이터로 나누기

In [ ]:
# 추가로 one-hot encoder 적용해보기
from sklearn.preprocessing import OneHotEncoder

# 특성과 반응변수 나누기
X = card_delete.drop(columns='credit')
y = card_delete['credit']

onehot = OneHotEncoder(sparse=False)
X_encoding = onehot.fit_transform(X)

In [ ]:
# 인코딩 한것을 적용하여 훈련데이터와 테스트 데이터로 나누기
from sklearn.model_selection import train_test_split


# 훈련 데이터와 검증 데이터로 나누기 --> credit의 개수를 계층적으로 분할
X_train,X_test,y_train,y_test = train_test_split(X_encoding,y,random_state=42,stratify = y, test_size = 0.25)

In [ ]:
from sklearn.model_selection import train_test_split

# 특성과 반응변수 나누기
X = card_delete.drop(columns='credit')
y = card_delete['credit']

# 훈련 데이터와 검증 데이터로 나누기 --> credit의 개수를 계층적으로 분할
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,stratify = y, test_size = 0.25)

In [ ]:
# gradient boosting을 통해 log-loss 확인해보기

from sklearn.metrics import accuracy_score, log_loss
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train,y_train)

y_pred = clf.predict_proba(X_test)
y_category = to_categorical(y_test)

log_loss(y_category,y_pred)

0.8532981230386966

In [ ]:
from sklearn.model_selection import GridSearchCV
clf

param_gird = {'learning_rate' : [0.01,0.1,1],
             'n_estimators' : [100,500,1000],
             'max_depth' :  [2,3,4,5] }

grid_search = GridSearchCV(GradientBoostingClassifier(),param_gird,cv=5)
grid_search.fit(X_train,y_train)

KeyboardInterrupt: ignored

In [ ]:
grid_search.best_params_

* svc 모델 사용해보기

In [ ]:
from sklearn.svm import SVC

# 객체 생성 및 학습
svc = SVC(probability=True)
svc.fit(X_train,y_train)

# 예측값 출력
y_pred = svc.predict_proba(X_test)
y_category = to_categorical(y_test)

# log-loss 구하기
print('log_loss : {:.3f}'.format(log_loss(y_category,y_pred)))

log_loss : 0.880


In [ ]:
# 스케일링 하고 결과 확인해보기
from sklearn.preprocessing import StandardScaler

# 객체 생성
scaler = StandardScaler()
scaler.fit(X_train)

# 스케일링 변환하기
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

# 모델 생성 및 학습
svc = SVC(probability=True)
svc.fit(X_train_trans,y_train)

# 예측값 출력
y_pred = svc.predict_proba(X_test_trans)
y_category = to_categorical(y_test)

# log-loss 구하기
print('log_loss : {:.3f}'.format(log_loss(y_category,y_pred)))

log_loss : 0.856


* LightGBM 사용해보기

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators = 500)
lgbm.fit(X_train,y_train,eval_metric = 'logloss',verbose=True)

y_pred = lgbm.predict_proba(X_test)
y_category = to_categorical(y_test)

print('log_loss : {:.3f}'.format(log_loss(y_category,y_pred)))

log_loss : 0.820


In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators = 500)

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]
        clf.fit(X_train, y_train)
        
        predictions=clf.predict_proba(X_test)
        
        logloss=log_loss(to_categorical(y_test), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(lgbm))

FOLD 0 : logloss:0.8150094254078722
FOLD 1 : logloss:0.8188992933546279
FOLD 2 : logloss:0.8076329237771621
FOLD 3 : logloss:0.7993166352457602
FOLD 4 : logloss:0.8058570239805448


0.8093430603531935

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators = 500)

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]
        clf.fit(X_train, y_train)
        
        predictions=clf.predict_proba(X_train)
        
        logloss=log_loss(to_categorical(y_train), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(lgbm))

FOLD 0 : logloss:0.5740185809153403
FOLD 1 : logloss:0.5777244294440868
FOLD 2 : logloss:0.5787635621781344
FOLD 3 : logloss:0.5780910868544654
FOLD 4 : logloss:0.5768726718339436


0.5770940662451942

In [ ]:
# scaling 하고 lgbm모델 적용해보기
from sklearn.model_selection import KFold, StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

lgbm = LGBMClassifier(n_estimators = 500)

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]

        # 스케일링 후 학습시키기
        scaler = StandardScaler().fit(X_train,y_train)
        X_train_trans = scaler.transform(X_train)
        X_test_trans = scaler.transform(X_test)

        clf.fit(X_train_trans, y_train)
        
        predictions=clf.predict_proba(X_test_trans)
        
        logloss=log_loss(to_categorical(y_test), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(lgbm))

FOLD 0 : logloss:0.8065903469567928
FOLD 1 : logloss:0.8015114389054364
FOLD 2 : logloss:0.7866310310222736
FOLD 3 : logloss:0.7903252808187003
FOLD 4 : logloss:0.7928047277943948


0.7955725650995196

In [ ]:
lgbm

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_gird = {'n_estimators' : [ 500, 700, 1000],
             'min_data_in_leaf' : [100,500,1000],
             'max_depth' :  [3,5,7,10] }

grid_search = GridSearchCV(LGBMClassifier(),param_gird,cv=5)
grid_search.fit(X_train,y_train)

grid_search.best_params_

{'max_depth': 10, 'min_data_in_leaf': 500, 'n_estimators': 700}

In [ ]:
# 최적 파라미터를 찾은 후의 결과 출력
lgbm = LGBMClassifier(n_estimators = 700,max_depth=10,min_data_in_leaf=500,num_leaves=128)
np.mean(run_kfold(lgbm))

FOLD 0 : logloss:0.834571123649263
FOLD 1 : logloss:0.8254208778761108
FOLD 2 : logloss:0.8249408426995714
FOLD 3 : logloss:0.8217601414141895
FOLD 4 : logloss:0.8243501060492021


0.8262086183376673

In [ ]:
# 보통 num_leaves = 2^(max_depth)는 depth-wise tree와 같은 수의 leaves를 가지게 하여, 이보다 작게 설정해야 오버피팅을 줄일 수 있다.

In [ ]:
train = card_delete

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [ ]:

lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.763423	valid_1's multi_logloss: 0.828584
[200]	training's multi_logloss: 0.71248	valid_1's multi_logloss: 0.814771
[300]	training's multi_logloss: 0.672563	valid_1's multi_logloss: 0.810216
[400]	training's multi_logloss: 0.636888	valid_1's multi_logloss: 0.806787
[500]	training's multi_logloss: 0.608141	valid_1's multi_logloss: 0.804994
Early stopping, best iteration is:
[524]	training's multi_logloss: 0.601899	valid_1's multi_logloss: 0.804391


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.763708	valid_1's multi_logloss: 0.831892
[200]	training's multi_logloss: 0.710956	valid_1's multi_logloss: 0.82201
[300]	training's multi_logloss: 0.669769	valid_1's multi_logloss: 0.815794
Ea

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

* Logitic 사용해보기

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler


lr = LogisticRegression()


def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]

        # 스케일링 후 학습시키기
        scaler = StandardScaler().fit(X_train,y_train)
        X_train_trans = scaler.transform(X_train)
        X_test_trans = scaler.transform(X_test)

        clf.fit(X_train_trans, y_train)
        
        predictions=clf.predict_proba(X_test_trans)
        
        logloss=log_loss(to_categorical(y_test), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(lr))

FOLD 0 : logloss:0.868236621485254
FOLD 1 : logloss:0.8658710641660343
FOLD 2 : logloss:0.8657941641035988
FOLD 3 : logloss:0.8655098182272697
FOLD 4 : logloss:0.8673076245804986


0.8665438585125311

* voting 방법 이용하기

In [ ]:
from sklearn.ensemble import VotingClassifier

# 모델 생성
gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,n_estimators=500)
svc = SVC(probability=True)
lgbm = LGBMClassifier(n_estimators=500)

# 객체 생성
voting = VotingClassifier(
    estimators = [('svc',svc),('LGBM',lgbm)], voting = 'soft'
)
voting.fit(X_train,y_train)

y_pred = voting.predict_proba(X_test)
y_category = to_categorical(y_test)

print('log_loss : {:.3f}'.format(log_loss(y_category,y_pred)))

In [ ]:
# 모델 생성
gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,n_estimators=500)
svc = SVC(probability=True)
lgbm = LGBMClassifier(n_estimators=500)

# 객체 생성
voting = VotingClassifier(estimators = [('svc',svc),('LGBM',lgbm)], voting = 'soft')

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]

        # 스케일링 후 학습시키기
        scaler = StandardScaler().fit(X_train,y_train)
        X_train_trans = scaler.transform(X_train)
        X_test_trans = scaler.transform(X_test)

        clf.fit(X_train_trans, y_train)
        
        predictions=clf.predict_proba(X_test_trans)
        
        logloss=log_loss(to_categorical(y_test), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(voting))

FOLD 0 : logloss:0.8062763541067263
FOLD 1 : logloss:0.801996965669735
FOLD 2 : logloss:0.7949928105963217
FOLD 3 : logloss:0.7973432371619573
FOLD 4 : logloss:0.8003844842510109


0.8001987703571503

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

# 모델 생성
gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,n_estimators=500)
#svc = SVC(probability=True)
lgbm = LGBMClassifier(n_estimators=500)

# 객체 생성
voting = VotingClassifier(
    estimators = [('gb',gb),('LGBM',lgbm)], voting = 'soft'
)
voting.fit(X_train,y_train)

y_pred = voting.predict_proba(X_test)
y_category = to_categorical(y_test)

print('log_loss : {:.3f}'.format(log_loss(y_category,y_pred)))

log_loss : 0.805


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

# 모델 생성
lr = LogisticRegression()
svc = SVC(probability=True)
lgbm = LGBMClassifier(n_estimators=500,learning_rate=0.05)

# 객체 생성
voting = VotingClassifier(estimators = [('svc',svc),('LGBM',lgbm)], voting = 'soft')

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]

        # 스케일링 후 학습시키기
        scaler = StandardScaler().fit(X_train,y_train)
        X_train_trans = scaler.transform(X_train)
        X_test_trans = scaler.transform(X_test)

        clf.fit(X_train_trans, y_train)
        
        predictions=clf.predict_proba(X_test_trans)
        
        logloss=log_loss(to_categorical(y_test), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(voting))

FOLD 0 : logloss:0.8179274057167527
FOLD 1 : logloss:0.8134220502725206
FOLD 2 : logloss:0.8087012142160184
FOLD 3 : logloss:0.8105079124884825
FOLD 4 : logloss:0.8130741154272574


0.8127265396242063

In [ ]:
# 로지스틱, svc , gradientboosting이용하여 voting 모델 만들기

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# 모델 생성
lr = LogisticRegression()
svc = SVC(probability=True)
gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,n_estimators=500)

# 객체 생성
voting = VotingClassifier(estimators = [('svc',svc),('lr',lr),('gb',gb)], voting = 'soft')

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]

        # 스케일링 후 학습시키기
        scaler = StandardScaler().fit(X_train,y_train)
        X_train_trans = scaler.transform(X_train)
        X_test_trans = scaler.transform(X_test)

        clf.fit(X_train_trans, y_train)
        
        predictions=clf.predict_proba(X_test_trans)
        
        logloss=log_loss(to_categorical(y_test), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes


np.mean(run_kfold(voting))

FOLD 0 : logloss:0.8250617246101
FOLD 1 : logloss:0.8247652392360025
FOLD 2 : logloss:0.8196095987163737
FOLD 3 : logloss:0.8195778199928055
FOLD 4 : logloss:0.8228668692851758


0.8223762503680915

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# 모델 생성
svc = SVC(probability=True)
gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,n_estimators=500)

# 객체 생성
voting = VotingClassifier(estimators = [('svc',svc),('gb',gb)], voting = 'soft')
np.mean(run_kfold(voting))

FOLD 0 : logloss:0.8201443292356998
FOLD 1 : logloss:0.8203563130600149
FOLD 2 : logloss:0.8133409329490885
FOLD 3 : logloss:0.8121243420342029
FOLD 4 : logloss:0.8168958641152707


0.8165723562788554

# ==========================================================

# test 데이터에 적용해보기

In [ ]:
# 테스트 데이터
card_test_delete

,gender,car,reality,child_num,income_type,edu_type,family_type,house_type,work_phone,phone,email,family_size,income_category,age_category,age_binary,employ,employ_year,employ_year_category,begin_year
0,1,1,0,0,1,4,0,1,0,1,0,2.0,1,6,1.0,0,0,0.0,5.0
1,0,0,1,0,2,1,1,1,0,1,0,2.0,2,5,1.0,1,24,1.0,3.0
2,0,0,1,0,4,4,1,1,1,1,0,2.0,1,4,1.0,1,1,0.0,3.0
3,1,1,0,0,0,4,1,1,1,0,0,2.0,1,5,1.0,1,7,0.0,3.0
4,0,1,1,0,2,1,1,1,1,0,0,2.0,3,4,1.0,1,26,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,0,4,2,1,1,1,1,0,2.0,3,5,1.0,1,15,1.0,1.0
9996,1,1,1,0,4,4,0,1,1,0,0,2.0,3,2,0.0,1,4,0.0,2.0
9997,0,0,1,0,4,4,1,1,0,0,0,2.0,4,5,1.0,1,39,2.0,4.0
9998,0,1,0,0,0,4,1,1,0,1,0,2.0,3,4,1.0,1,3,0.0,2.0


In [ ]:
#  훈련 데이터 전체를 학습시키기 

from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X,y)

y_pred = clf.predict_proba(card_test_delete)
y_pred

In [ ]:
#  훈련 데이터 전체를 학습시키기 
from sklearn.ensemble import VotingClassifier

# 모델 생성
#gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,n_estimators=500)
svc = SVC(probability=True)
lgbm = LGBMClassifier(n_estimators=1000)

scaler = StandardScaler().fit(X,y)
X_trans = scaler.transform(X)
X_test_trans = scaler.transform(card_test_delete)

# 객체 생성
voting = VotingClassifier(
    estimators = [('svc',svc),('LGBM',lgbm)], voting = 'soft'
)
voting.fit(X_trans,y)

y_pred = voting.predict_proba(X_test_trans)

result = pd.DataFrame(y_pred)

NameError: ignored

In [ ]:
result.to_csv('submission.csv')

In [ ]:
#  훈련 데이터 전체를 학습시키기 
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators = 700,max_depth=10,min_data_in_leaf=500,num_leaves=128)
lgbm.fit(X,y,eval_metric = 'logloss',verbose=True)

y_pred = lgbm.predict_proba(card_test_delete)
result = pd.DataFrame(y_pred)
result.to_csv('submission1.csv')

# 중복 데이터를 제외하고 데이터 학습시켜보기

In [ ]:
# begin_month 와 credit를 제외한 중복 행들 제거한 후 모델 적용해보기
mask = np.where(card.drop(columns='credit').duplicated()==False)[0]
mask

array([    0,     1,     2, ..., 26454, 26455, 26456])

In [ ]:
card_mask = card.loc[mask,:]
# 위의 조건들을 삭제한 변수
card_mask_delete = card_mask.drop(columns=['income_total','DAYS_BIRTH','DAYS_EMPLOYED','employ_day','unemploy_day','begin_month','unemploy','age'])
card_mask_delete

,gender,car,reality,child_num,income_type,edu_type,family_type,house_type,work_phone,phone,email,family_size,credit,income_category,age_category,age_binary,employ,employ_year,employ_year_category,begin_year
0,0,0,0,0,0,1,1,2,0,0,0,2.0,1.0,3,3,0.0,1,13,1.0,0.0
1,0,0,1,1,0,4,0,1,0,0,1,3.0,1.0,4,3,0.0,1,5,0.0,0.0
2,1,1,1,0,4,1,1,1,0,1,0,2.0,2.0,4,5,1.0,1,13,1.0,1.0
3,0,0,1,0,0,4,1,1,0,1,0,2.0,0.0,3,4,1.0,1,6,0.0,3.0
4,0,1,1,0,2,1,1,1,0,0,0,2.0,2.0,2,4,1.0,1,6,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,2,4,1,1,0,0,0,4.0,1.0,3,3,0.0,1,6,0.0,0.0
26453,0,0,1,1,4,1,2,1,0,0,0,2.0,2.0,3,4,1.0,1,7,0.0,3.0
26454,0,1,0,0,4,4,0,5,0,0,0,2.0,2.0,4,2,0.0,1,6,0.0,2.0
26455,1,0,1,0,4,2,3,1,0,0,0,1.0,2.0,3,2,0.0,1,1,0.0,4.0


In [ ]:
# 데이터 나누기
from sklearn.model_selection import train_test_split

# 특성과 반응변수 나누기
X = card_mask_delete.drop(columns='credit')
y = card_mask_delete['credit']

# 훈련 데이터와 검증 데이터로 나누기 --> credit의 개수를 계층적으로 분할
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,stratify = y, test_size = 0.25)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from lightgbm import LGBMClassifier

clf = LGBMClassifier(n_estimators = 700,max_depth=10,min_data_in_leaf=500,num_leaves=128)
def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]
        clf.fit(X_train, y_train)
        
        predictions=clf.predict_proba(X_test)
        
        logloss=log_loss(to_categorical(y_test), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(clf))

FOLD 0 : logloss:0.8279435649150445
FOLD 1 : logloss:0.8331194903113709
FOLD 2 : logloss:0.8279501553422527
FOLD 3 : logloss:0.8312706778543915
FOLD 4 : logloss:0.8291193254069241


0.8298806427659967

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from lightgbm import LGBMClassifier

clf = LGBMClassifier(n_estimators=500)

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    #sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[val_index]
        y_train, y_test = y.iloc[train_index], y.iloc[val_index]
        clf.fit(X_train, y_train)
        
        predictions=clf.predict_proba(X_train)
        
        logloss=log_loss(to_categorical(y_train), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
    return outcomes

np.mean(run_kfold(clf))

FOLD 0 : logloss:0.5638406872121341
FOLD 1 : logloss:0.5620645892286278
FOLD 2 : logloss:0.5617743288006338
FOLD 3 : logloss:0.563425847953891
FOLD 4 : logloss:0.5616695033417124


0.5625549913073998

* test data를 넣어서 모델 적용하기


In [ ]:
# 특성과 반응변수 나누기 --> 훈련데이터
X = card_mask_delete.drop(columns='credit')
y = card_mask_delete['credit']

In [ ]:
# 테스트 데이터
card_test_delete

,gender,car,reality,child_num,income_type,edu_type,family_type,house_type,work_phone,phone,email,family_size,income_category,age_category,employ,employ_year,begin_year,age_binary,employ_year_category
0,1,1,0,0,1,4,0,1,0,1,0,2.0,1,6,0,0,5.0,1.0,0.0
1,0,0,1,0,2,1,1,1,0,1,0,2.0,2,5,1,24,3.0,1.0,1.0
2,0,0,1,0,4,4,1,1,1,1,0,2.0,1,4,1,1,3.0,1.0,0.0
3,1,1,0,0,0,4,1,1,1,0,0,2.0,1,5,1,7,3.0,1.0,0.0
4,0,1,1,0,2,1,1,1,1,0,0,2.0,3,4,1,26,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,0,4,2,1,1,1,1,0,2.0,3,5,1,15,1.0,1.0,1.0
9996,1,1,1,0,4,4,0,1,1,0,0,2.0,3,2,1,4,2.0,0.0,0.0
9997,0,0,1,0,4,4,1,1,0,0,0,2.0,4,5,1,39,4.0,1.0,2.0
9998,0,1,0,0,0,4,1,1,0,1,0,2.0,3,4,1,3,2.0,1.0,0.0


In [ ]:
#  훈련 데이터 전체를 학습시키기 
from sklearn.ensemble import VotingClassifier

# 모델 생성
#gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,n_estimators=500)
svc = SVC(probability=True)
lgbm = LGBMClassifier(n_estimators=1000)

scaler = StandardScaler().fit(X,y)
X_trans = scaler.transform(X)
X_test_trans = scaler.transform(card_test_delete)

# 객체 생성
voting = VotingClassifier(
    estimators = [('svc',svc),('LGBM',lgbm)], voting = 'soft'
)
voting.fit(X_trans,y)

y_pred = voting.predict_proba(X_test_trans)

result = pd.DataFrame(y_pred)

In [ ]:
result.to_csv('submission.csv')

# 처음부터 그냥 원핫 인코딩만 해보기

In [ ]:
train = card
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = card_test
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

object_col

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type']

In [ ]:
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

train.head()

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_NAN,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff
0,0,202500.0,-13899,-4709,1,0,0,0,2.0,-6.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,247500.0,-11380,-1540,1,0,0,1,3.0,-5.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,450000.0,-19087,-4434,1,0,1,0,2.0,-22.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,202500.0,-15088,-2092,1,0,1,0,2.0,-37.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,157500.0,-15037,-2105,1,0,0,0,2.0,-26.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [ ]:
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators = 500)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.677988	valid_1's multi_logloss: 0.759487
[200]	training's multi_logloss: 0.60364	valid_1's multi_logloss: 0.740355
[300]	training's multi_logloss: 0.54788	valid_1's multi_logloss: 0.73406
[400]	training's multi_logloss: 0.502351	valid_1's multi_logloss: 0.729556
Early stopping, best iteration is:
[435]	training's multi_logloss: 0.488139	valid_1's multi_logloss: 0.728254


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.677007	valid_1's multi_logloss: 0.769037
[200]	training's multi_logloss: 0.601918	valid_1's multi_logloss: 0.755076
[300]	training's multi_logloss: 0.545834	valid_1's multi_logloss: 0.749569
[400]	training's multi_logloss: 0.498598	valid_1's multi_logloss: 0.746432
Ear

In [ ]:
lgb_models

{0: LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=700, n_jobs=-1, num_leaves=31, objective=None,
                random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 1: LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=700, n_jobs=-1, num_leaves=31, objective=None,
                random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 2: LGBMClassifier(boosting_type='gbdt', class_weight=None, co

In [ ]:
submit = pd.read_csv('submission.csv')
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [ ]:
submit.to_csv('submission1.csv')

# 새 섹션